In [224]:
# utilities
import re
import numpy as np
import pandas as pd

# nltk
from nltk.stem import WordNetLemmatizer

#SpellCorrection
from spellchecker import SpellChecker

import string
import emoji
import chardet

In [203]:
DATASET_COLUMNS = ['date', 'username', 'text', 'polarity', 'emotion']

#Detect file encoding using chardet
with open('data.csv', 'rb') as f:
    result = chardet.detect(f.read())

# Print the detected encoding
print("Detected encoding:", result['encoding'])

# Read the file using the detected encoding
df = pd.read_csv('data.csv', encoding=result['encoding'], names=DATASET_COLUMNS)
df.sample(5)

Detected encoding: UTF-8-SIG


,date,username,text,polarity,emotion
31,"10:05 AM · Oct 25, 2023",@cryptounitrade,✅ #PUMP $TOMO! +0.796% in 15 seconds\nCurrent ...,1,happy
241,"6:25 PM · Oct 25, 2023",@CryptoScamNever,"""🚨 Attention all crypto enthusiasts! Stay one ...",-1,fear
179,"12:33 AM · Oct 26, 2023",\n@yevano__oruvan,Here are my drawings! Let's take a look and le...,1,happy
57,"7:31 AM · Oct 24, 2023",@CallBotCrypto,How could AI foresee this? #BTC\n\n📊📈Inside th...,1,surprise
292,"12:01 PM · Oct 25, 2023\n",@Jyo5200,📈 Just found this old screenshot of #Bitcoin ...,1,surprise


In [204]:
#Data preprocessing
data=df[['text','polarity', 'emotion']]

In [205]:
data['polarity'].unique()

array([ 1, -1,  0])

In [206]:
data_pos = data[data['polarity'] == 1]
data_neu = data[data['polarity'] == 0]
data_neg = data[data['polarity'] == -1]

In [207]:
data_pos = data_pos.iloc[:int(100)]
data_neu = data_neu.iloc[:int(100)]
data_neg = data_neg.iloc[:int(100)] 

In [208]:
dataset = pd.concat([data_pos, data_neu, data_neg])

In [209]:
def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)
dataset['text'] = dataset['text'].apply(lambda x: cleaning_numbers(x))
dataset['text'].tail()

247    #alts , after nailing the whole of it .i think...
261    Be cautious with unofficial apps and websites ...
277    Cryptocurrency lending firm BlockFi has declar...
285    Ffs #cryptocurrency stop give me - more hours ...
287    Bad News for the #crypto market.\n\n📌The ticke...
Name: text, dtype: object

In [210]:
emoticons_to_keep = [
    '💰', '📈', '🤣', '🎊', '😂', '😭', '🙁', '😞', '💔', '😢', '😮', '😵', '🙀',
    '😱', '❗', '😠', '😡', '😤', '👎', '🔪', '🌕', '🚀', '💎', '👀', '💭', '📉',
    '😨', '😩', '😰', '💸'
]

def clean_tweet(text):
    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)

    # Remove hashtags and mentions
    text = re.sub(r'@\w+|#\w+', '', text)

    # Remove special characters except for emoticons
    text = re.sub(r'[^\w\s.!?{}]+'.format(''.join(emoticons_to_keep)), '', text)

    # Remove extra whitespace
    text = ' '.join(text.split())

    return text

# Apply the modified cleaning function to the 'text' column in your dataset
dataset['text'] = dataset['text'].apply(clean_tweet)

# Display the 'text' column in the entire dataset
print(dataset['text'])

0      BTC ON GLP RESISTANCE FOR NOW PLAY SAFE IF U R...
1            HAY bullflag breakout👀 Lets fill that wick🚀
2      Did you guys see how is doing a pitch with a d...
3      GN Fam going early to bed been up since or AM ...
4      You think this week has been fun?!? 😂😂😂😂 Wait ...
                             ...                        
247    after nailing the whole of it .i think its tim...
261    Be cautious with unofficial apps and websites ...
277    Cryptocurrency lending firm BlockFi has declar...
285    Ffs stop give me more hours to accumulate jesu...
287    Bad News for the market. The ticker for BlackR...
Name: text, Length: 147, dtype: object


In [211]:
# Initialize SpellChecker only once to avoid re-creation for each call
spell = SpellChecker()

# Function for spell correction
def spell_correction(text):
    words = text.split()
    misspelled = spell.unknown(words)
    corrected_words = []
    for word in words:
        if word in misspelled:
            corrected_word = spell.correction(word)
            # Check if the correction is not None, otherwise use the original word
            corrected_words.append(corrected_word if corrected_word is not None else word)
        else:
            corrected_words.append(word)
    return ' '.join(corrected_words)

# Apply spell correction to the entire 'text' column
dataset['text'] = dataset['text'].apply(spell_correction)

# Display the entire dataset
print(dataset)

                                                  text  polarity       emotion
0    BTC ON GLP RESISTANCE FOR NOW PLAY SAFE IF U R...         1         happy
1            HAY bullfrog breakout Lets fill that wick         1  anticipation
2    Did you guys see how is doing a pitch with a d...         1         happy
3    GN Fam going early to bed been up since or AM ...         1         happy
4    You think this week has been fun?!? 😂😂😂😂 Wait ...         1  anticipation
..                                                 ...       ...           ...
247  after nailing the whole of it i think its time...        -1          fear
261  Be cautious with unofficial apps and websites ...        -1          fear
277  Cryptocurrency lending firm BlockFi has declar...        -1           sad
285  Ffs stop give me more hours to accumulate jesu...        -1         angry
287  Bad News for the market The ticker for BlackRo...        -1           sad

[147 rows x 3 columns]


In [212]:
#Define the emoticon dictionary outside the function for a wider scope
emoticon_dict = {
    ":)": "smile ",
    ":(": "sad ",
    ":D": "laugh ",
    "😊": "smiling face with smiling eyes ",
    "😃": "grinning face with big eyes ",
    "😉": "winking face ",
    "👌": "OK hand ",
    "👍": "Thumbs up ",
    "😁": "beaming face with smiling eyes ",
    "😂": "face with tears of joy ",
    "😄": "grinning face with smiling eyes ",
    "😅": "grinning face with sweat ",
    "😆": "grinning squinting face ",
    "😇": "smiling face with halo ",
    "😞": "disappointed face ",
    "😔": "pensive face ",
    "😑": "expressionless face ",
    "😒": "unamused face ",
    "😓": "downcast face with sweat ",
    "😕": "confused face ",
    "😖": "confounded face ",
    "💰": "Money Bag ",
    "📈": "Up Trend ",
    "🤣": "Rolling on the Floor Laughing ",
    "🎊": "Confetti Ball ",
    "😭": "Loudly Crying ",
    "🙁": "Slightly frowning face ",
    "💔": "Broken Heart ",
    "😢": "Crying Face ",
    "😮": "Face with Open Mouth ",
    "😵": "Dizzy Face ",
    "🙀": "Weary Cat ",
    "😱": "Face Screaming in Fear ",
    "❗": "Exclamation Mark ",
    "😠": "Angry Face ",
    "😡": "Pouting Face ",
    "😤": "Face with Steam from Nose ",
    "👎": "Thumbs Down ",
    "🔪": "Hocho ",
    "🌕": "Moon ",
    "🚀": "Rocket ",
    "💎": "Diamond ",
    "👀": "Eyes ",
    "💭": "Thought Balloon ",
    "📉": "Down Trend ",
    "😨": "Fearful Face ",
    "😩": "Weary Face ",
    "😰": "Anxious Face with Fear ",
    "💸": "Money with Wings "
}

# Emoticon to word conversion function
def convert_emoticons_to_words(text):
    changed_emoticons = 0  # Variable to count the number of changed emoticons
    for emoticon, word in emoticon_dict.items():
        while emoticon in text:
            text = text.replace(emoticon, word + " ", 1)
            changed_emoticons += 1
    return text, changed_emoticons

# Apply the function and count emoticons for each row
def apply_conversion(text):
    converted_text, count = convert_emoticons_to_words(text)
    return pd.Series([converted_text, count], index=['converted_text', 'emoticons_count'])

conversion_results = dataset['text'].apply(apply_conversion)
dataset['converted_text'] = conversion_results['converted_text']
dataset['emoticons_count'] = conversion_results['emoticons_count']
print("Emoticons converted to words in 'converted_text' column.")
print(dataset[['converted_text', 'emoticons_count']].head())

Emoticons converted to words in 'converted_text' column.
                                      converted_text  emoticons_count
0  BTC ON GLP RESISTANCE FOR NOW PLAY SAFE IF U R...                0
1          HAY bullfrog breakout Lets fill that wick                0
2  Did you guys see how is doing a pitch with a d...                0
3  GN Fam going early to bed been up since or AM ...                0
4  You think this week has been fun?!? face with ...               13


In [213]:
stopwordlist = ['a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
             'and','any','are', 'as', 'at', 'be', 'because', 'been', 'before',
             'being', 'below', 'between','both', 'by', 'can', 'd', 'did', 'do',
             'does', 'doing', 'down', 'during', 'each','few', 'for', 'from',
             'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
             'into','is', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma',
             'me', 'more', 'most','my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'only', 'or', 'other', 'our', 'ours','ourselves', 'out', 'own', 're','s', 'same', 'she', "shes", 'should', "shouldve",'so', 'some', 'such',
             't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those',
             'through', 'to', 'too','under', 'until', 'up', 've', 'very', 'was',
             'we', 'were', 'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will', 'with', 'won', 'y', 'you', "youd","youll", "youre",
             "youve", 'your', 'yours', 'yourself', 'yourselves']

In [214]:
# Stopwords removal applied separately after the option has been chosen and processed
STOPWORDS = set(stopwordlist)
def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

# Apply the stopwords cleaning after the loop, once the 'text' column has been updated accordingly
dataset['text'] = dataset['converted_text'].apply(cleaning_stopwords)
print("Stopwords removed from 'text' column.")
print(dataset['text'].head())

Stopwords removed from 'text' column.
0    BTC ON GLP RESISTANCE FOR NOW PLAY SAFE IF U R...
1                 HAY bullfrog breakout Lets fill wick
2    Did guys see pitch deck reaching community Tha...
3    GN Fam going early bed since AM morning nonsto...
4    You think week fun?!? face tears joy face tear...
Name: text, dtype: object


In [215]:
# Function to clean repeating words
def cleaning_repeating_words(text):
    # This regex pattern targets whole words that are repeated
    return re.sub(r'\b(\w+)( \1\b)+', r'\1', text)

# Assuming 'dataset' is a pandas DataFrame and 'text' is a column in it
# Apply the cleaning function for repeating words to each row in the 'text' column
dataset['text'] = dataset['text'].apply(cleaning_repeating_words)
print("Repeating words cleaned from 'text' column.")
print(dataset['text'].head())

Repeating words cleaned from 'text' column.
0    BTC ON GLP RESISTANCE FOR NOW PLAY SAFE IF U R...
1                 HAY bullfrog breakout Lets fill wick
2    Did guys see pitch deck reaching community Tha...
3    GN Fam going early bed since AM morning nonsto...
4    You think week fun?!? face tears joy face tear...
Name: text, dtype: object


In [216]:
dataset['text']=dataset['text'].str.lower()
dataset['text'].head()

0    btc on glp resistance for now play safe if u r...
1                 hay bullfrog breakout lets fill wick
2    did guys see pitch deck reaching community tha...
3    gn fam going early bed since am morning nonsto...
4    you think week fun?!? face tears joy face tear...
Name: text, dtype: object

In [217]:
from nltk.tokenize import RegexpTokenizer

# The pattern matches word characters (\w) and punctuation marks ([^\w\s])
tokenizer = RegexpTokenizer(r'\w+|[^\w\s]')

# Applying the modified tokenizer to the dataset
dataset['text'] = dataset['text'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
dataset['text'] = dataset['text'].apply(tokenizer.tokenize)
dataset['text'].head()

0    [btc, on, glp, resistance, for, now, play, saf...
1          [hay, bullfrog, breakout, lets, fill, wick]
2    [did, guys, see, pitch, deck, reaching, commun...
3    [gn, fam, going, early, bed, since, am, mornin...
4    [you, think, week, fun, ?, !, ?, face, tears, ...
Name: text, dtype: object

In [218]:
import nltk
st = nltk.PorterStemmer()
def stemming_on_text(data):
    text = [st.stem(word) for word in data]
    return data
dataset['text']= dataset['text'].apply(lambda x: stemming_on_text(x))
dataset['text'].head()

0    [btc, on, glp, resistance, for, now, play, saf...
1          [hay, bullfrog, breakout, lets, fill, wick]
2    [did, guys, see, pitch, deck, reaching, commun...
3    [gn, fam, going, early, bed, since, am, mornin...
4    [you, think, week, fun, ?, !, ?, face, tears, ...
Name: text, dtype: object

In [219]:
lm = nltk.WordNetLemmatizer()
def lemmatizer_on_text(data):
    text = [lm.lemmatize(word) for word in data]
    return data
dataset['text'] = dataset['text'].apply(lambda x: lemmatizer_on_text(x))
dataset['text'].head()

0    [btc, on, glp, resistance, for, now, play, saf...
1          [hay, bullfrog, breakout, lets, fill, wick]
2    [did, guys, see, pitch, deck, reaching, commun...
3    [gn, fam, going, early, bed, since, am, mornin...
4    [you, think, week, fun, ?, !, ?, face, tears, ...
Name: text, dtype: object

In [220]:
# All necessary imports should be placed at the top of the script
import numpy as np
from keras.models import Sequential, Model
from keras.layers import LSTM, Dense, Embedding, Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Initialize the tokenizer with your dataset
# NOTE: Ensure `dataset` is correctly loaded with 'text' column
tokenizer = Tokenizer()
tokenizer.fit_on_texts(dataset['text'])

# Function to create LSTM model
def create_lstm_model(input_length, num_classes):
    model = Sequential()
    model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=input_length))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# Function to extract features from LSTM model
def extract_features(model, data):
    intermediate_layer_model = Model(inputs=model.input, outputs=model.layers[-2].output)
    return intermediate_layer_model.predict(data)

# Assuming `dataset` is a pandas DataFrame with 'text', 'polarity', and 'emotion' columns
sequences = tokenizer.texts_to_sequences(dataset['text'])
X = pad_sequences(sequences, maxlen=50)

# Convert polarity and emotion labels to one-hot encoding
encoder_polarity = LabelEncoder()
y_polarity = to_categorical(encoder_polarity.fit_transform(dataset['polarity']))

encoder_emotion = LabelEncoder()
y_emotion = to_categorical(encoder_emotion.fit_transform(dataset['emotion']))

# Split the data into training and testing sets for polarity and emotion
X_train_polarity, X_test_polarity, y_train_polarity, y_test_polarity = train_test_split(X, y_polarity, test_size=0.2, random_state=42)
X_train_emotion, X_test_emotion, y_train_emotion, y_test_emotion = train_test_split(X, y_emotion, test_size=0.2, random_state=42)

# Create and train LSTM models for polarity and emotion
lstm_model_polarity = create_lstm_model(50, y_polarity.shape[1])
lstm_model_emotion = create_lstm_model(50, y_emotion.shape[1])

lstm_model_polarity.fit(X_train_polarity, y_train_polarity, epochs=5, batch_size=32)
lstm_model_emotion.fit(X_train_emotion, y_train_emotion, epochs=5, batch_size=32)

Epoch 1/5
4/4 [==============================] - 2s 43ms/step - loss: 1.0962 - accuracy: 0.4017
Epoch 2/5
4/4 [==============================] - 0s 42ms/step - loss: 1.0624 - accuracy: 0.7009
Epoch 3/5
4/4 [==============================] - 0s 32ms/step - loss: 1.0157 - accuracy: 0.6752
Epoch 4/5
4/4 [==============================] - 0s 31ms/step - loss: 0.9133 - accuracy: 0.6752
Epoch 5/5
4/4 [==============================] - 0s 40ms/step - loss: 0.8016 - accuracy: 0.6752
Epoch 1/5
4/4 [==============================] - 2s 24ms/step - loss: 1.9412 - accuracy: 0.1966
Epoch 2/5
4/4 [==============================] - 0s 27ms/step - loss: 1.8993 - accuracy: 0.4615
Epoch 3/5
4/4 [==============================] - 0s 28ms/step - loss: 1.8140 - accuracy: 0.4701
Epoch 4/5
4/4 [==============================] - 0s 31ms/step - loss: 1.6543 - accuracy: 0.3504
Epoch 5/5
4/4 [==============================] - 0s 30ms/step - loss: 1.5442 - accuracy: 0.3419


In [221]:
# Extract features with LSTM for SVM training
X_train_features_polarity = extract_features(lstm_model_polarity, X_train_polarity)
X_train_features_emotion = extract_features(lstm_model_emotion, X_train_emotion)

# Train SVM for polarity and emotion
svm_classifier_polarity = SVC(kernel='linear', probability=True)
svm_classifier_emotion = SVC(kernel='linear', probability=True)

svm_classifier_polarity.fit(X_train_features_polarity, np.argmax(y_train_polarity, axis=1))
svm_classifier_emotion.fit(X_train_features_emotion, np.argmax(y_train_emotion, axis=1))

4/4 [==============================] - 0s 9ms/step


SVC(kernel='linear', probability=True)

In [223]:
# Assuming `polarity_labels` is your list of original labels for the training data
# Example list of polarity labels used in your training dataset
polarity_labels = ['positive', 'negative', 'neutral']  # This should be replaced by the actual labels you have

# Initialize the LabelEncoder
encoder_polarity = LabelEncoder()

# Fit the LabelEncoder with your actual labels
encoder_polarity.fit(polarity_labels)

# Assuming encoder is a pre-defined LabelEncoder object for decoding the emotion labels

# This function should extract LSTM features
def extract_features(model, sequence):
    intermediate_layer_model = Model(inputs=model.input, outputs=model.layers[-2].output)
    return intermediate_layer_model.predict(sequence)

# Function to classify the intensity
def classify_intensity(emoticons_count, text):
    question_marks = text.count('?')
    periods = text.count('.')
    exclamation_marks = text.count('!')

    if exclamation_marks > 1 or question_marks > 1 or emoticons_count > 1:
        return 'High'
    elif periods == 1 or question_marks == 1 or emoticons_count == 1 or exclamation_marks ==1 :
        return 'Medium'
    elif question_marks == 0 and emoticons_count == 0:
        return 'Low'
    else:
        return 'Undetermined'

def tokenize_text(text):
    tokenizer = RegexpTokenizer(r'\w+|[^\w\s]')
    return ' '.join(tokenizer.tokenize(text))

#Function to perform real-time prediction and intensity classification
def real_time_prediction(text, tokenizer, lstm_model_emotion, lstm_model_polarity, svm_classifier_emotion, svm_classifier_polarity, encoder_emotion, encoder_polarity):
    # Preprocessing steps (assuming these functions are defined elsewhere in your code)
    
    # Assuming we have a single text input and all necessary models, tokenizers, and encoders
    tweet = "I'm happy!!!"

    cleaned_text = cleaning_numbers(text)
    cleaned_tweet = clean_tweet(cleaned_text)
    corrected_text = spell_correction(cleaned_tweet)
    emoticon_converted_text, emoticons_count = convert_emoticons_to_words(corrected_text)  # Ensure this function returns emoticons_count
    cleaned_stopwords = cleaning_stopwords(emoticon_converted_text)
    cleaned_repeating_words = cleaning_repeating_words(cleaned_stopwords)

    # Now tokenize the text after cleaning repeating words
    tokenized_text = tokenize_text(cleaned_repeating_words)

    # Continue with any additional preprocessing steps that work on the tokenized text
    stemmed_text = stemming_on_text(tokenized_text)
    lemmatized_text = lemmatizer_on_text(stemmed_text)

    assert isinstance(lemmatized_text, str), "Processed text must be a string"

    # Convert the processed text to a sequence
    sequence = tokenizer.texts_to_sequences([lemmatized_text])
    padded_sequence = pad_sequences(sequence, maxlen=50)


    # Predict emotion and polarity using the LSTM model
    lstm_prediction_emotion = lstm_model_emotion.predict(padded_sequence)
    lstm_features_emotion = extract_features(lstm_model_emotion, padded_sequence)
    svm_prediction_emotion = svm_classifier_emotion.predict_proba(lstm_features_emotion)

    lstm_prediction_polarity = lstm_model_polarity.predict(padded_sequence)
    lstm_features_polarity = extract_features(lstm_model_polarity, padded_sequence)
    svm_prediction_polarity = svm_classifier_polarity.predict_proba(lstm_features_polarity)

    # Decode the predicted labels
    emotion_label = encoder_emotion.inverse_transform(np.argmax(lstm_prediction_emotion, axis=1))
    polarity_label = encoder_polarity.inverse_transform(np.argmax(svm_prediction_polarity, axis=1))

    # Get probabilities for the predicted labels
    emotion_probability = np.max(lstm_prediction_emotion, axis=1)
    polarity_probability = np.max(svm_prediction_polarity, axis=1)

    # Classify the intensity
    intensity = classify_intensity(emoticons_count, text)  # Ensure `emoticons_count` is defined

    return polarity_label, emotion_label, polarity_probability, emotion_probability, intensity


# Call the real-time prediction function
polarity_label, emotion_label, polarity_probability, emotion_probability, intensity = real_time_prediction(
    tweet, tokenizer, lstm_model_emotion, lstm_model_polarity, svm_classifier_emotion, svm_classifier_polarity, encoder_emotion, encoder_polarity)

# Print the results
print(f"Tweet: {tweet}")
print(f"Polarity Label: {polarity_label[0]}")
print(f"Emotion Label: {emotion_label[0]}")
print(f"Intensity Level: {intensity}")

1/1 [==============================] - 0s 370ms/step
Tweet: I'm angry!!!
Polarity Label: negative
Emotion Label: anticipation
Intensity Level: High
